In [146]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Specify each path

In [147]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.keras'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [148]:
NUM_CLASSES = 8

# Dataset reading

In [149]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [150]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [152]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [153]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

model.save('model/keypoint_classifier/keypoint_classifier.keras')

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dropout_16 (Dropout)                 │ (None, 42)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 20)                  │             860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 10)                  │             210 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 8)                   │              88 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,158 (4.52 KB)

 Trainable params: 1,158 (4.52 KB)

 Non-trainable params: 0 (0.00 B)

In [154]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [155]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [156]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.0781 - loss: 2.2666
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.0931 - loss: 2.2320 - val_accuracy: 0.1256 - val_loss: 2.0928
Epoch 2/1000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0938 - loss: 2.2083
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1269 - loss: 2.1361 - val_accuracy: 0.1628 - val_loss: 2.0314
Epoch 3/1000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1328 - loss: 2.0718
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1304 - loss: 2.0594 - val_accuracy: 0.1953 - val_loss: 1.9938
Epoch 4/1000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.1406 - loss: 2.0328
Epoch 4: saving model to model/keypoint_classifier/keypoint_classifier.ker

In [157]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9596 - loss: 0.2190


In [158]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [159]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
[1.2913548e-05 4.1249211e-04 1.5602791e-01 2.6419589e-02 1.5737549e-04
 1.3513131e-03 8.1561679e-01 1.6382011e-06]
6


# Confusion matrix

In [160]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, classification_report

# def print_confusion_matrix(y_true, y_pred, report=True):
#     labels = sorted(list(set(y_true)))
#     cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
#     df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
#     fig, ax = plt.subplots(figsize=(7, 6))
#     sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
#     ax.set_ylim(len(set(y_true)), 0)
#     plt.show()
    
#     if report:
#         print('Classification Report')
#         print(classification_report(y_test, y_pred))

# Y_pred = model.predict(X_test)
# y_pred = np.argmax(Y_pred, axis=1)

# print_confusion_matrix(y_test, y_pred)

# Convert to model for Tensorflow-Lite

In [161]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [162]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\Pc\AppData\Local\Temp\tmpze73jec1\assets


INFO:tensorflow:Assets written to: C:\Users\Pc\AppData\Local\Temp\tmpze73jec1\assets


Saved artifact at 'C:\Users\Pc\AppData\Local\Temp\tmpze73jec1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 42), dtype=tf.float32, name='input_layer_8')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2728272916560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2728652705040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2728652716176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2728652704848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2728652704464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2728652703696: TensorSpec(shape=(), dtype=tf.resource, name=None)


6748

# Inference test

In [163]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [164]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [165]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [166]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: total: 0 ns
Wall time: 0 ns


In [167]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[1.2913561e-05 4.1249214e-04 1.5602782e-01 2.6419591e-02 1.5737565e-04
 1.3513138e-03 8.1561685e-01 1.6382012e-06]
6
